In [1]:
import pygame
import sys
import numpy as np
import random
import pandas as pd
import time
import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)



pygame 2.1.2 (SDL 2.0.18, Python 3.7.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
width=10
height=10
pixel_size=(50,50)
display_size=(pixel_size[0]*width,pixel_size[1]*height)
record=[]
game_num=1
p=0
step=0
BATCH_SIZE=128
gamma=0.95
reward=[0]

color_dict={
    "wall":(177,177,177),
    "target":(0,177,0),
    "normal":(255,255,255),
    "start":(177,0,0)
}

map_dict={
    "target":(7,2),
    "start":(2,height-1),
    "tank1":(7,4),
    "tank2":(5,2),
    "tank3":(random.randint(3,width-3),random.randint(2,height-3)),

   }

wall_location=[]
for w in range(width):
    wall_location.append((w,0))
    wall_location.append((w,height-1))
    
for h in range(height):    
    wall_location.append((0,h))
    wall_location.append((width-1,h))
    
wall_location.remove(map_dict.get("start"))
wall_location.append((map_dict.get("start")[0],map_dict.get("start")[1]+1))
map_dict['wall']=wall_location


df=pd.DataFrame(columns=['s1','s2','a','s1_','s2_','r'])


a_dict={
    0:'up',
    1:'down',
    2:'left',
    3:'right'
}

s_index={}
for i in range(width-2):
    for j in range(height-2):
        s_index.update({(i+1,j+1):(i+(j*(width-2)))})
s_index.update({map_dict.get("start"):(width-2)*(height-2)})   


In [3]:
def draw_map():
    for i in range(width*height):
        w=i-width*(i//width)
        h=i//width
        
        if (w,h)==map_dict.get("target"):
            color=color_dict.get("target")
        elif (w,h)==map_dict.get("start"):
            color=color_dict.get("start")
        elif (w,h) in map_dict.get("wall"):
            color=color_dict.get("wall")
        else:
            color=color_dict.get("normal")
        
        pygame.draw.rect(game_map,color,(w*50,h*50,pixel_size[0],pixel_size[1]))

 

In [4]:
class tank(pygame.sprite.Sprite):
    def __init__(self,filename,location):
        pygame.sprite.Sprite.__init__(self)
        self.image = pygame.image.load(filename).convert()
        self.image = pygame.transform.scale(self.image,(50,50))

        self.rect = self.image.get_rect()
        self.rect.topleft=(location[0]*pixel_size[0],location[1]*pixel_size[1])
        

def crash_detect(sprite1,sprite2):
    global game_num
    global game_state
    result=pygame.sprite.collide_mask(sprite1,sprite2)
    
    if result:
        if sprite2 in tank_group:
            print("这是第{}局游戏，我死了".format(game_num))
            if game_num>20:
                print('最后20盘的胜率为{}'.format(record[-20:].count(1)/20))
            record.append(0)
            reward.append(-1)

        elif sprite2 in princess_group:
            print("这是第{}局游戏，我赢了".format(game_num))
            if game_num>20:
                print('最后20盘的胜率为{}'.format(record[-20:].count(1)/20))
            record.append(1)
            reward.append(10)


        game_num+=1
        game_state=1


def crash_wall():
    if (person_site[0],person_site[1]) in map_dict.get("wall"):
        #reward.append(-1.0)
        return True
    else:
        #reward.append(-0.02)
        return False
    
    
def move(direction):
    global person_site
    global step
    if direction=='up':
        person_site[1]-=1
        a= crash_wall()
        if a==True:
            person_site[1]+=1
        
    if direction=='down':
        person_site[1]+=1
        a= crash_wall()
        if a==True:
            person_site[1]-=1
    if direction=='left':
        person_site[0]-=1
        a= crash_wall()
        if a==True:
            person_site[0]+=1
    if direction=='right':
        person_site[0]+=1
        a= crash_wall()
        if a==True:
            person_site[0]-=1
        
    person.rect=(person_site[0]*pixel_size[0],person_site[1]*pixel_size[1],50,50)
    crash_detect(person,tank1)       
    crash_detect(person,tank2)   
    crash_detect(person,tank3)       

    crash_detect(person,princess)         
    step+=1

    
def reset():
    global person_site
    global game_state
    person_site=[map_dict.get("start")[0],map_dict.get("start")[1]]
    person.rect=(person_site[0]*pixel_size[0],person_site[1]*pixel_size[1],50,50)
    game_state=0
    reward.append(-0.02)
    
    if game_num>70 and game_num%20==0:    
        pgdebug()   

In [5]:
def pick_a(state):    
    p=model.predict([state])
    a=np.random.choice(p.shape[1],p=p.ravel())
    return a
        
    
def trace_collect(s1,s2,a,s1_,s2_,r):   
    global df
    new_trace={
        's1':s1,
        's2':s2,
        'a':a,
        's1_':s1_,
        's2_':s2_,
        'r':r
    }
    df=df.append(new_trace,ignore_index=True)
    
    
def preprocessing():
    global df
    train_ds=df
    dr=0
    n=len(train_ds['r'])
    for t in range(n):
        k=n-t-1
        dr=dr*gamma+train_ds['r'][k]
        train_ds['r'][k]=dr    
        
    df=pd.DataFrame(columns=['s1','s2','a','s1_','s2_','r'])
    #print(train_ds)
    return train_ds


def train_step(train_ds):
    global BATCH_SIZE
    BATCH_SIZE=len(train_ds)
    s = np.int32(np.stack((train_ds['s1'],train_ds['s2']),1))
    a = np.array(train_ds['a'])
    s_ = np.int32(np.stack((train_ds['s1_'],train_ds['s2_']),1))
    r = np.array(train_ds['r'])
    
    

   
    with tf.GradientTape(persistent=True) as tape:
        #loss函数
        q=model(s)        
        loss=tf.keras.losses.sparse_categorical_crossentropy(a,q)        
        loss=r*loss        
        
    gradient=tape.gradient(loss,model.trainable_variables)
    opt.apply_gradients(zip(gradient,model.trainable_variables))
    del tape
    
    
def one_turn():
    s1=int(person_site[0])
    s2=int(person_site[1])
    a=pick_a([s1,s2])
    move(a_dict.get(a))
    s1_=int(person_site[0])
    s2_=int(person_site[1])
    r=reward[-1]
    trace_collect(s1,s2,a,s1_,s2_,r)
                
    if game_state==1:          
        train_ds=preprocessing()
        train_step(train_ds)
        reset()


In [6]:
model=tf.keras.Sequential([
    tf.keras.layers.Dense(128,input_shape=(BATCH_SIZE,2),activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(4,activation='softmax')       #a,qmax=r+gamma*q_max
   ]
)


opt=tf.keras.optimizers.Adam(learning_rate=0.0001)

In [7]:
def pgdebug():
    for i in range(width-2):
        for j in range(height-2):
            result=model.predict([[i+1,j+1]])
            print(i+1,j+1)
            print(result)


In [8]:
pygame.init()
clock = pygame.time.Clock()
pygame.display.set_caption("AI训练游戏")

screen = pygame.display.set_mode(display_size)
game_map=pygame.surface.Surface(display_size)
draw_map()

tank_group = pygame.sprite.Group()
princess_group = pygame.sprite.Group()
person_group=pygame.sprite.Group()

tank1 = tank("img/tank.jpg",map_dict.get("tank1"))
tank2 = tank("img/tank.jpg",map_dict.get("tank2"))
tank3 = tank("img/tank.jpg",map_dict.get("tank3"))

tank_group.add(tank1)
tank_group.add(tank2)
tank_group.add(tank3)


princess=tank('img/sidalin.jpeg',map_dict.get("target"))
princess_group.add(princess)

person_site=[map_dict.get("start")[0],map_dict.get("start")[1]]
person=tank('img/maintank.jpeg',person_site)
person_group.add(person)

reset()

while True:
    clock.tick(60)

    screen.blit(game_map,(0,0))
    tank_group.draw(screen)
    princess_group.draw(screen)
    screen.blit(person.image,person.rect)
    
    
    
    one_turn()



        
    for event in pygame.event.get():
        if event.type ==pygame.QUIT:
            pygame.quit()
            sys.exit()
            

        
#       if event.type ==pygame.KEYDOWN:
#           if event.key == pygame.K_UP:
#               move('up')
#           if event.key == pygame.K_DOWN:
#               move('down')
#           if event.key == pygame.K_LEFT:
#               move('left')
#           if event.key == pygame.K_RIGHT:
#               move('right')
#           if event.key == pygame.K_SPACE:
                
                
     
    pygame.display.update()

这是第1局游戏，我死了
这是第2局游戏，我死了
这是第3局游戏，我死了
这是第4局游戏，我死了
这是第5局游戏，我死了
这是第6局游戏，我死了
这是第7局游戏，我死了
这是第8局游戏，我赢了
这是第9局游戏，我死了
这是第10局游戏，我死了
这是第11局游戏，我死了
这是第12局游戏，我死了
这是第13局游戏，我死了
这是第14局游戏，我死了
这是第15局游戏，我死了
这是第16局游戏，我死了
这是第17局游戏，我死了
这是第18局游戏，我死了
这是第19局游戏，我死了
这是第20局游戏，我死了
这是第21局游戏，我死了
最后20盘的胜率为0.05
这是第22局游戏，我死了
最后20盘的胜率为0.05
这是第23局游戏，我死了
最后20盘的胜率为0.05
这是第24局游戏，我死了
最后20盘的胜率为0.05
这是第25局游戏，我死了
最后20盘的胜率为0.05
这是第26局游戏，我死了
最后20盘的胜率为0.05
这是第27局游戏，我死了
最后20盘的胜率为0.05
这是第28局游戏，我死了
最后20盘的胜率为0.05
这是第29局游戏，我死了
最后20盘的胜率为0.0
这是第30局游戏，我死了
最后20盘的胜率为0.0
这是第31局游戏，我死了
最后20盘的胜率为0.0
这是第32局游戏，我死了
最后20盘的胜率为0.0
这是第33局游戏，我死了
最后20盘的胜率为0.0
这是第34局游戏，我死了
最后20盘的胜率为0.0
这是第35局游戏，我死了
最后20盘的胜率为0.0
这是第36局游戏，我死了
最后20盘的胜率为0.0
这是第37局游戏，我死了
最后20盘的胜率为0.0
这是第38局游戏，我死了
最后20盘的胜率为0.0
这是第39局游戏，我死了
最后20盘的胜率为0.0
这是第40局游戏，我死了
最后20盘的胜率为0.0
这是第41局游戏，我死了
最后20盘的胜率为0.0
这是第42局游戏，我死了
最后20盘的胜率为0.0
这是第43局游戏，我死了
最后20盘的胜率为0.0
这是第44局游戏，我死了
最后20盘的胜率为0.0
这是第45局游戏，我死了
最后20盘的胜率为0.0
这是第46局游戏，我赢了
最后20盘的胜率为0.0
这是第47局游戏，我死了
最后20盘的胜率为0.05
这是第48局游戏，我死了
最后20盘的胜率为0.05
这是第49局游戏，我死

6 5
[[0.2707193  0.21894267 0.25412625 0.25621182]]
6 6
[[0.27811944 0.2181779  0.2580183  0.24568434]]
6 7
[[0.2831517  0.2162848  0.25908697 0.24147654]]
6 8
[[0.29011136 0.21362257 0.2566388  0.23962723]]
7 1
[[0.25749087 0.21180445 0.22802599 0.3026787 ]]
7 2
[[0.2603921  0.20964497 0.23003776 0.29992518]]
7 3
[[0.2615307  0.20920266 0.23551369 0.2937529 ]]
7 4
[[0.26278123 0.21121472 0.2404314  0.28557265]]
7 5
[[0.26747653 0.21198048 0.24949892 0.2710441 ]]
7 6
[[0.27525097 0.21484762 0.25559852 0.2543029 ]]
7 7
[[0.28234836 0.21348657 0.2592536  0.2449115 ]]
7 8
[[0.2874052  0.21163511 0.26035565 0.240604  ]]
8 1
[[0.25771147 0.20715119 0.2246999  0.31043744]]
8 2
[[0.26089382 0.20502105 0.22591105 0.308174  ]]
8 3
[[0.26219815 0.20385453 0.23095699 0.30299035]]
8 4
[[0.26333356 0.20511483 0.23596586 0.29558575]]
8 5
[[0.2652912  0.20702477 0.24226533 0.2854187 ]]
8 6
[[0.2715968  0.20767923 0.2513427  0.2693813 ]]
8 7
[[0.27962694 0.21062875 0.25704733 0.25269696]]
8 8
[[0.2866

7 6
[[0.30182168 0.18312794 0.21716219 0.2978882 ]]
7 7
[[0.30622056 0.18235257 0.2241223  0.28730452]]
7 8
[[0.31011915 0.18014884 0.23024446 0.27948752]]
8 1
[[0.2792022  0.19155195 0.18387529 0.34537056]]
8 2
[[0.28264588 0.18642944 0.18233617 0.34858856]]
8 3
[[0.28952828 0.18135355 0.18421143 0.3449067 ]]
8 4
[[0.2953806  0.17761566 0.18882419 0.33817956]]
8 5
[[0.29693013 0.17823826 0.19552729 0.3293043 ]]
8 6
[[0.30238283 0.17584705 0.20614658 0.31562352]]
8 7
[[0.30925733 0.17526591 0.21358328 0.30189344]]
8 8
[[0.31356877 0.17432867 0.22040565 0.2916969 ]]
这是第140局游戏，我死了
最后20盘的胜率为0.3
这是第141局游戏，我死了
最后20盘的胜率为0.3
这是第142局游戏，我赢了
最后20盘的胜率为0.25
这是第143局游戏，我赢了
最后20盘的胜率为0.3
这是第144局游戏，我死了
最后20盘的胜率为0.35
这是第145局游戏，我死了
最后20盘的胜率为0.3
这是第146局游戏，我死了
最后20盘的胜率为0.25
这是第147局游戏，我死了
最后20盘的胜率为0.25
这是第148局游戏，我赢了
最后20盘的胜率为0.25
这是第149局游戏，我死了
最后20盘的胜率为0.25
这是第150局游戏，我死了
最后20盘的胜率为0.25
这是第151局游戏，我死了
最后20盘的胜率为0.25
这是第152局游戏，我死了
最后20盘的胜率为0.25
这是第153局游戏，我死了
最后20盘的胜率为0.25
这是第154局游戏，我赢了
最后20盘的胜率为0.2
这是第155局游戏，我死了

8 7
[[0.33761904 0.08620229 0.12092219 0.45525655]]
8 8
[[0.34315977 0.08265227 0.12125604 0.45293188]]
这是第180局游戏，我死了
最后20盘的胜率为0.2
这是第181局游戏，我死了
最后20盘的胜率为0.2
这是第182局游戏，我死了
最后20盘的胜率为0.2
这是第183局游戏，我死了
最后20盘的胜率为0.2
这是第184局游戏，我死了
最后20盘的胜率为0.2
这是第185局游戏，我死了
最后20盘的胜率为0.2
这是第186局游戏，我赢了
最后20盘的胜率为0.2
这是第187局游戏，我死了
最后20盘的胜率为0.25
这是第188局游戏，我赢了
最后20盘的胜率为0.2
这是第189局游戏，我死了
最后20盘的胜率为0.2
这是第190局游戏，我死了
最后20盘的胜率为0.2
这是第191局游戏，我赢了
最后20盘的胜率为0.2
这是第192局游戏，我死了
最后20盘的胜率为0.25
这是第193局游戏，我死了
最后20盘的胜率为0.25
这是第194局游戏，我死了
最后20盘的胜率为0.2
这是第195局游戏，我死了
最后20盘的胜率为0.2
这是第196局游戏，我死了
最后20盘的胜率为0.2
这是第197局游戏，我死了
最后20盘的胜率为0.2
这是第198局游戏，我死了
最后20盘的胜率为0.2
这是第199局游戏，我死了
最后20盘的胜率为0.2
1 1
[[0.2816204  0.1985617  0.22245876 0.29735914]]
1 2
[[0.28914264 0.18703757 0.21893205 0.30488774]]
1 3
[[0.2983452  0.17363557 0.21223484 0.31578442]]
1 4
[[0.30747947 0.160832   0.20509027 0.32659823]]
1 5
[[0.31641793 0.14853255 0.19778548 0.337264  ]]
1 6
[[0.3251124  0.13703926 0.19062278 0.34722555]]
1 7
[[0.3335695  0.12615195 0.18357755 0.

这是第233局游戏，我死了
最后20盘的胜率为0.25
这是第234局游戏，我死了
最后20盘的胜率为0.2
这是第235局游戏，我死了
最后20盘的胜率为0.15
这是第236局游戏，我死了
最后20盘的胜率为0.15
这是第237局游戏，我赢了
最后20盘的胜率为0.15
这是第238局游戏，我死了
最后20盘的胜率为0.2
这是第239局游戏，我死了
最后20盘的胜率为0.15
1 1
[[0.2888328  0.19368272 0.23050816 0.2869763 ]]
1 2
[[0.30092442 0.18047693 0.22566414 0.29293454]]
1 3
[[0.31315178 0.16711895 0.21995008 0.29977918]]
1 4
[[0.3259738  0.15418166 0.21380438 0.30604017]]
1 5
[[0.33855012 0.14167418 0.20742439 0.31235132]]
1 6
[[0.35082632 0.12970766 0.20075026 0.31871575]]
1 7
[[0.36304548 0.11842643 0.19378024 0.32474783]]
1 8
[[0.37498897 0.10787468 0.18668383 0.33045256]]
2 1
[[0.3027479  0.17201973 0.21898864 0.3062437 ]]
2 2
[[0.31471753 0.1590635  0.21549542 0.31072354]]
2 3
[[0.32609305 0.1479224  0.209797   0.31618756]]
2 4
[[0.33820713 0.13678311 0.204574   0.3204358 ]]
2 5
[[0.34999654 0.12563336 0.19793417 0.32643586]]
2 6
[[0.3617953  0.11555073 0.19156349 0.33109048]]
2 7
[[0.37400225 0.10608479 0.18491966 0.3349933 ]]
2 8
[[0.3861302  0.0966757

...

4 6
[[2.3109531e-01 4.0256526e-04 1.5419334e-03 7.6696014e-01]]
4 7
[[2.1455574e-01 2.0079843e-04 8.6997566e-04 7.8437352e-01]]
4 8
[[2.02386439e-01 1.00624864e-04 4.92718362e-04 7.97020197e-01]]
5 1
[[0.05852187 0.2560922  0.3143686  0.3710173 ]]
5 2
[[0.7212287  0.00299663 0.01611871 0.25965595]]
5 3
[[7.9416388e-01 4.4458092e-04 3.1057079e-03 2.0228581e-01]]
5 4
[[6.3766479e-01 3.3939077e-04 1.9562726e-03 3.6003956e-01]]
5 5
[[3.9937061e-01 3.0149924e-04 1.4110859e-03 5.9891683e-01]]
5 6
[[2.4748223e-01 2.0157396e-04 8.8586926e-04 7.5143039e-01]]
5 7
[[2.0008536e-01 1.0618310e-04 5.1749754e-04 7.9929096e-01]]
5 8
[[1.7772295e-01 5.2469575e-05 2.9081778e-04 8.2193381e-01]]
6 1
[[0.04236605 0.26586583 0.3101203  0.3816478 ]]
6 2
[[0.2418986  0.0397993  0.19475667 0.5235455 ]]
6 3
[[8.4151268e-01 2.1727258e-04 2.0952520e-03 1.5617481e-01]]
6 4
[[8.1171715e-01 9.9734221e-05 9.4840355e-04 1.8723467e-01]]
6 5
[[6.2194782e-01 8.5911088e-05 6.5015513e-04 3.7731612e-01]]
6 6
[[3.8568583e-01 

4 6
[[0.20579308 0.0011741  0.00339146 0.78964126]]
4 7
[[1.8782264e-01 6.4798846e-04 2.0615556e-03 8.0946779e-01]]
4 8
[[1.7486960e-01 3.6326755e-04 1.2676520e-03 8.2349950e-01]]
5 1
[[0.04369813 0.2694518  0.264572   0.42227808]]
5 2
[[0.0575198  0.1925053  0.32263276 0.42734212]]
5 3
[[0.6597311  0.00230243 0.01148501 0.32648146]]
5 4
[[0.5623053  0.00121922 0.00525918 0.43121627]]
5 5
[[0.32422206 0.00106758 0.003596   0.6711144 ]]
5 6
[[2.1280646e-01 7.3833694e-04 2.3330941e-03 7.8412211e-01]]
5 7
[[1.7264511e-01 3.8192724e-04 1.3355855e-03 8.2563740e-01]]
5 8
[[1.5244411e-01 2.0494591e-04 7.9034531e-04 8.4656060e-01]]
6 1
[[0.02983228 0.27063578 0.2544255  0.44510636]]
6 2
[[0.03548725 0.23532216 0.28726825 0.4419223 ]]
6 3
[[0.3804017  0.01141981 0.06242197 0.5457565 ]]
6 4
[[7.1098745e-01 5.0685054e-04 3.4501059e-03 2.8505564e-01]]
6 5
[[5.3461105e-01 3.9359604e-04 2.0936129e-03 4.6290168e-01]]
6 6
[[3.004017e-01 3.419149e-04 1.435533e-03 6.978209e-01]]
6 7
[[1.8789366e-01 2.39

4 6
[[1.3782457e-01 2.7397613e-04 9.9356496e-04 8.6090797e-01]]
4 7
[[1.19616896e-01 1.27969703e-04 5.23242168e-04 8.79731894e-01]]
4 8
[[1.05588764e-01 6.03487424e-05 2.75705650e-04 8.94075096e-01]]
5 1
[[0.0836549  0.20791563 0.33542898 0.3730005 ]]
5 2
[[0.6921233  0.00226331 0.01016207 0.29545128]]
5 3
[[0.5651395  0.00091775 0.00394035 0.4300024 ]]
5 4
[[3.0816171e-01 6.1016018e-04 2.0790296e-03 6.8914908e-01]]
5 5
[[1.7729084e-01 3.4534154e-04 1.1678956e-03 8.2119590e-01]]
5 6
[[1.3440168e-01 1.5696100e-04 6.1763526e-04 8.6482370e-01]]
5 7
[[1.0424709e-01 6.6610766e-05 3.0308997e-04 8.9538312e-01]]
5 8
[[8.7781891e-02 3.0384801e-05 1.5710668e-04 9.1203064e-01]]
6 1
[[0.06459304 0.214037   0.33264238 0.3887276 ]]
6 2
[[0.682015   0.00338092 0.01583178 0.29877234]]
6 3
[[6.8283284e-01 4.1032568e-04 2.4416281e-03 3.1431517e-01]]
6 4
[[4.8804826e-01 2.5260943e-04 1.2774562e-03 5.1042163e-01]]
6 5
[[2.4350083e-01 1.6240199e-04 6.6860626e-04 7.5566816e-01]]
6 6
[[1.4105976e-01 8.650124

SystemExit: 

d:\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
